# MultiQC Jupyter Notebook Example

This notebook has some example code showing how MultiQC can be used within an interactive analysis, such as a Jupyter Notebook.

MultiQC is written in Python, so must be used in a Python environment. If possible, Python 3.5 or later is recommended. MultiQC can be installed in a variety of ways: see [the documentation](https://multiqc.info/docs/#installing-multiqc) for more information. Note that MultiQC must be installed into the notebook _kernal_.

First, let's install MultiQC using pip (note the `%` magic which installs the package into the kernel and not the jupyter environment).

> NB: Support for imports is only available from v1.8 of MultiQC. At the time of writing this is not yet released, so we will install the development version directly from GitHub.

In [1]:
%pip install --upgrade git+https://github.com/ewels/MultiQC.git

  Cloning https://github.com/ewels/MultiQC.git to /private/var/folders/tk/k7tjvpqs0tbfd0bzvt4htrzh0000gn/T/pip-req-build-_gly3u7p
  Running command git clone -q https://github.com/ewels/MultiQC.git /private/var/folders/tk/k7tjvpqs0tbfd0bzvt4htrzh0000gn/T/pip-req-build-_gly3u7p
  Running command git submodule update --init --recursive -q
  Stored in directory: /private/var/folders/tk/k7tjvpqs0tbfd0bzvt4htrzh0000gn/T/pip-ephem-wheel-cache-jp0doza9/wheels/6b/46/2f/3efddef001e00a278ecc722a2caa45b408c684b363ce55678b
Successfully built multiqc
  Found existing installation: multiqc 1.8.dev0
ERROR: Exception:
Traceback (most recent call last):
  File "/Users/philewels/miniconda2/envs/altair/lib/python3.6/site-packages/pip/_internal/cli/base_command.py", line 178, in main
    status = self.run(options, args)
  File "/Users/philewels/miniconda2/envs/altair/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 414, in run
    use_user_site=options.use_user_site,
  File "/Users/phi

> NB: You will probably need to restart the notebook kernal after installing MultiQC

Now let's import the `multiqc` package into your workbook:

In [2]:
import multiqc

Great! Now let's check that it's working properly by printing the version that we're using:

In [3]:
print(multiqc.__version__)

1.8.dev0 (25aafd9)


Before we can use any outputs from MultiQC, we must first run it on some data. The [MultiQC website](https://multiqc.info/) has the logs used for all of the example reports on the homepage available for download, so let's grab the files for the RNA-seq report.

In [4]:
!wget https://multiqc.info/examples/rna-seq/data.zip
!unzip -o data.zip
!rm data.zip

--2019-11-11 15:13:41--  https://multiqc.info/examples/rna-seq/data.zip
Resolving multiqc.info (multiqc.info)... 91.238.163.174
Connecting to multiqc.info (multiqc.info)|91.238.163.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18549188 (18M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  17.69M  7.59MB/s    in 2.3s    

2019-11-11 15:13:43 (7.59 MB/s) - ‘data.zip’ saved [18549188/18549188]

Archive:  data.zip
  inflating: data/fastqc_theoretical_gc_hg38_txome.txt  
  inflating: data/SRR3192396_1.fastq.gz_trimming_report.txt  
  inflating: data/SRR3192396_1_fastqc.html  
  inflating: data/SRR3192396_1_fastqc.zip  
  inflating: data/SRR3192396_1_star_aligned.bam_counts.txt.summary  
  inflating: data/SRR3192396_1_val_1_fastqc.html  
  inflating: data/SRR3192396_1_val_1_fastqc.zip  
  inflating: data/SRR3192396_1Log.final.out  
  inflating: data/SRR3192396_1Log.out  
  inflating: data/SRR3192396_1Log.progress.out  


You should now see a folder called `data/` in your notebook work directory with a bunch of log files within from a typical RNA-seq analysis run.

Now let's run MultiQC on those files:

In [5]:
multiqc.run('./data/')

Usage: ipykernel_launcher.py [OPTIONS] <analysis directory>
Try "ipykernel_launcher.py -h" for help.

Error: Invalid value for "<analysis directory>": Path "d" does not exist.


SystemExit: 2

/Users/philewels/miniconda2/envs/altair/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
